# Explore the stream at Embudo NM

In [61]:
%matplotlib inline

import gzip

import matplotlib.pyplot as plt
import numpy as np
import spacepy.toolbox as tb
import spacepy.plot as spp
import pandas as pd



In [78]:
dfile = 'Embudo_stream_ts_raw.txt.gz'

In [82]:
dat = pd.read_csv(dfile, skiprows=30, delimiter='\t', header=None, names=('agency', 'site','datetime', 'TZ', 
                                                                          'discharge', 'd_a', 'gague', 'g_a'))
print(dat[0:20])


   agency     site          datetime   TZ  discharge d_a  gague g_a
0    USGS  8279500  2007-10-01 00:15  MDT      536.0   A   3.25   A
1    USGS  8279500  2007-10-01 00:30  MDT      536.0   A   3.25   A
2    USGS  8279500  2007-10-01 00:45  MDT      536.0   A   3.25   A
3    USGS  8279500  2007-10-01 01:00  MDT      531.0   A   3.24   A
4    USGS  8279500  2007-10-01 01:15  MDT      531.0   A   3.24   A
5    USGS  8279500  2007-10-01 01:30  MDT      531.0   A   3.24   A
6    USGS  8279500  2007-10-01 01:45  MDT      531.0   A   3.24   A
7    USGS  8279500  2007-10-01 02:00  MDT      531.0   A   3.24   A
8    USGS  8279500  2007-10-01 02:15  MDT      526.0   A   3.23   A
9    USGS  8279500  2007-10-01 02:30  MDT      526.0   A   3.23   A
10   USGS  8279500  2007-10-01 02:45  MDT      526.0   A   3.23   A
11   USGS  8279500  2007-10-01 03:00  MDT      526.0   A   3.23   A
12   USGS  8279500  2007-10-01 03:15  MDT      522.0   A   3.22   A
13   USGS  8279500  2007-10-01 03:30  MDT      5

In [83]:
index = pd.DatetimeIndex(dat['datetime'], tz=dat['TZ'])

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [87]:
dat['TZ'].shape


(308222,)